In [2]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../src')

import pandas as pd
pd.set_option('display.max_rows', 100)


from sklearn.metrics import mean_squared_error

from sales_project.utils.plotters import linear_plot
from sales_project.utils.common import save_predictions
from sales_project.tuners import run_optuna, time_series_objective

from tqdm.auto import tqdm

In [1]:
import cudf
from cuml.ensemble import RandomForestRegressor as cuRF
from cuml.linear_model import ElasticNet as cuElasticNet
from cuml.linear_model import Ridge as cuRidge
from cuml.linear_model import Lasso as cuLasso
from cuml.svm import LinearSVR as cuLinearSVR
from cuml.svm import SVR as cuSVR
from cuml.preprocessing import RobustScaler, MinMaxScaler


In [3]:
# Reading data
df_train = cudf.read_csv('../data/artifacts/df_monthly3.csv')
df_submission = cudf.read_csv('../data/artifacts/df_submission.csv')

total_size = len(df_train)

# Converting date column's type to datetime
# df_train['date'] = cudf.to_datetime(df_train['date'])
# df_train.set_index('date', inplace=True)

# Train test split
df_test = df_train[df_train['date_block_num'] > 32]
df_train = df_train[df_train['date_block_num'] <= 32]

df_train.drop(['shop_id', 'item_id'], axis=1, inplace=True)
df_test.drop(['shop_id', 'item_id'], axis=1, inplace=True)
df_submission.drop(['date_block_num'], axis=1, inplace=True)

print(f"Train data shape: {df_train.shape}")
print(f"Test data shape: {df_test.shape}")
print(f"Ratio of train data: {df_train.shape[0] / total_size}")
print(f"Ratio of test data: {df_test.shape[0] / total_size}")

# Features and target
features = list(df_train.columns)
target = 'item_cnt_month'
timestamp_col = 'date_block_num'
features.remove(target)
features.remove(timestamp_col)
print(f'Features: {features}')

Train data shape: (2496501, 95)
Test data shape: (103209, 95)
Ratio of train data: 0.9602998026702978
Ratio of test data: 0.03970019732970216
Features: ['last_item_price', 'last_item_revenue_day', 'last_log1p_item_price', 'last_log1p_item_revenue_day', 'last_norm_diff_cat_price', 'last_norm_diff_cat_revenue', 'last_norm_diff_cat_log1p_price', 'last_norm_diff_cat_log1p_revenue', 'item_name_emb_0', 'item_name_emb_1', 'item_name_cleaned_length', 'item_name_length', 'item_category_name_emb_0', 'item_category_name_emb_1', 'item_category_name_cleaned_length', 'item_category_name_length', 'shop_name_emb_0', 'shop_name_emb_1', 'item_cnt_month.lag.1', 'item_cnt_month.lag.2', 'item_cnt_month.lag.3', 'item_revenue_month.lag.1', 'item_revenue_month.lag.2', 'item_revenue_month.lag.3', 'item_name_group.item_cnt_month.sum.lag.1', 'item_name_group.item_cnt_month.mean.lag.1', 'item_id.item_cnt_month.mean.lag.1', 'platform_id.item_cnt_month.mean.lag.1', 'supercategory_id.item_cnt_month.mean.lag.1', 'ite

In [4]:
# Scaling data (since linear models are sensitive to the scaling factor)

for col in tqdm(features):
    scaler = MinMaxScaler()
    df_train[[col]] = scaler.fit_transform(df_train[[col]])
    df_test[[col]] = scaler.transform(df_test[[col]])
    df_submission[[col]] = scaler.transform(df_submission[[col]])

  0%|          | 0/93 [00:00<?, ?it/s]

In [5]:
for col in tqdm(features):
    df_train[col] = df_train[col].astype('float32')
    df_test[col] = df_test[col].astype('float32')
    df_submission[col] = df_submission[col].astype('float32')

  0%|          | 0/93 [00:00<?, ?it/s]

#### Lasso -> Val: 0.98415, Test: 0.79666, Kaggle: 1.12898

In [6]:
# Setting the grid for hyperparameter optimization
hyperparameters_grid = {
    "alpha": {
        "method": "suggest_loguniform",
        "kwargs": {"low": 1e-8, "high": 1e2},
    }
}

# Optimizing via Optuna
best_model, test_scores = run_optuna(
    df_train=df_train,
    df_test=df_test,
    features=features,
    target=target,
    timestamp_col='date_block_num',
    objective=time_series_objective,
    scorer=lambda y, pred: mean_squared_error(y.to_numpy(), pred.to_numpy())**(0.5),
    cv='per_timestamp',
    val_ratio=0.1,
    direction="minimize",
    n_trials=100,
    model_class = cuLasso,
    hyperparameters_grid=hyperparameters_grid,
    is_gpu_accelerated=True,
    step=2,
)
print(test_scores)

# Predicting and saving predictions for the submission subset of the data
df_submission['item_cnt_month'] = best_model.predict(df_submission[features]).clip(lower=0)
save_predictions(df_submission[['shop_id', 'item_id', 'item_cnt_month']].to_pandas(), 'lasso_1.csv')

[I 2024-08-09 19:52:53,479] A new study created in memory with name: no-name-c61beecb-92e7-4be2-bccb-f855890334f0
[I 2024-08-09 19:53:05,568] Trial 0 finished with value: 0.9841865348499809 and parameters: {'alpha': 1.8980615118687408e-06}. Best is trial 0 with value: 0.9841865348499809.
[I 2024-08-09 19:53:09,130] Trial 1 finished with value: 1.3212327740670602 and parameters: {'alpha': 2.946479738621991}. Best is trial 0 with value: 0.9841865348499809.
[I 2024-08-09 19:53:16,739] Trial 2 finished with value: 0.9961493067616816 and parameters: {'alpha': 0.00018803596878259543}. Best is trial 0 with value: 0.9841865348499809.
[I 2024-08-09 19:53:20,346] Trial 3 finished with value: 1.3212327740670602 and parameters: {'alpha': 3.5169793843699013}. Best is trial 0 with value: 0.9841865348499809.
[I 2024-08-09 19:53:30,031] Trial 4 finished with value: 0.9845772280459644 and parameters: {'alpha': 7.93467707041097e-06}. Best is trial 0 with value: 0.9841865348499809.
[I 2024-08-09 19:53:41

Best hyperparameters:  {'alpha': 4.859893039425955e-07}
{'MAE': 0.32406305525552637, 'MSE': 0.634660757827184, 'RMSE': 0.7966559846176918, 'R2': 0.42253514250530255, 'MAPE': 722660965587138.8, 'SMAPE': nan}


#### Elastic Net -> Val: 0.98547, Test: 0.79420, Kaggle: 1.06627

In [6]:
# Setting the grid for hyperparameter optimization
hyperparameters_grid = {
    "alpha": {
        "method": "suggest_loguniform",
        "kwargs": {"low": 1e-5, "high": 1e2},
    },
    "l1_ratio": {
        "method": "suggest_uniform",
        "kwargs": {"low": 0, "high": 1},        
    }
}

# Optimizing via Optuna
best_model, test_scores = run_optuna(
    df_train=df_train,
    df_test=df_test,
    features=features,
    target=target,
    timestamp_col='date_block_num',
    objective=time_series_objective,
    scorer=lambda y, pred: mean_squared_error(y.to_numpy(), pred.to_numpy())**(0.5),
    cv='per_timestamp',
    val_ratio=0.1,
    direction="minimize",
    n_trials=100,
    model_class = cuElasticNet,
    hyperparameters_grid=hyperparameters_grid,
    is_gpu_accelerated=True,
    step=2,
)
print(test_scores)

# Predicting and saving predictions for the submission subset of the data
df_submission['item_cnt_month'] = best_model.predict(df_submission[features]).clip(lower=0)
save_predictions(df_submission[['shop_id', 'item_id', 'item_cnt_month']].to_pandas(), 'elastic_net_2.csv')

[I 2024-08-09 19:18:56,685] A new study created in memory with name: no-name-35bf71bb-fab1-4936-bf25-1cd4a53eb0e8
[I 2024-08-09 19:19:04,208] Trial 0 finished with value: 1.2637672804466589 and parameters: {'alpha': 0.15881992773128153, 'l1_ratio': 0.013400480792112979}. Best is trial 0 with value: 1.2637672804466589.
[I 2024-08-09 19:19:20,767] Trial 1 finished with value: 1.0475252463885343 and parameters: {'alpha': 0.00045562298320430664, 'l1_ratio': 0.23212769952167533}. Best is trial 1 with value: 1.0475252463885343.
[I 2024-08-09 19:19:26,487] Trial 2 finished with value: 1.2245424907681306 and parameters: {'alpha': 0.018331110307819146, 'l1_ratio': 0.7316702133577491}. Best is trial 1 with value: 1.0475252463885343.
[I 2024-08-09 19:19:30,266] Trial 3 finished with value: 1.3212327740670602 and parameters: {'alpha': 0.15837381052180755, 'l1_ratio': 0.9590265811434331}. Best is trial 1 with value: 1.0475252463885343.
[I 2024-08-09 19:19:42,070] Trial 4 finished with value: 1.0157

Best hyperparameters:  {'alpha': 1.5636216470528864e-05, 'l1_ratio': 0.9991259587450578}
{'MAE': 0.323295400454367, 'MSE': 0.6307604527610383, 'RMSE': 0.7942042890598352, 'R2': 0.42608395040216795, 'MAPE': 719329796693737.1, 'SMAPE': nan}


#### SVM (linear) -> Val: 0.92500, Test: 0.80325 Kaggle: 1.08980

In [6]:
# Setting the grid for hyperparameter optimization
hyperparameters_grid = {
    "epsilon": {
        "method": "suggest_loguniform",
        "kwargs": {"low": 1e-2, "high": 1e+0},
    },
    "C": {
        "method": "suggest_loguniform",
        "kwargs": {"low": 1e-4, "high": 1e+3},
    },
    "penalty": {
        "method": "suggest_categorical",
        "kwargs": {"choices": ['l1', 'l2']},
    },
    "verbose": {
        "method": "suggest_categorical",
        "kwargs": {"choices": [0]},
    }
}

# Optimizing via Optuna
best_model, test_scores = run_optuna(
    df_train=df_train,
    df_test=df_test,
    features=features,
    target=target,
    timestamp_col='date_block_num',
    objective=time_series_objective,
    scorer=lambda y, pred: mean_squared_error(y.to_numpy(), pred.to_numpy())**(0.5),
    cv='per_timestamp',
    val_ratio=0.1,
    direction="minimize",
    n_trials=100,
    model_class = cuLinearSVR,
    hyperparameters_grid=hyperparameters_grid,
    is_gpu_accelerated=True,
    step=5,
)
print(test_scores)

# Predicting and saving predictions for the submission subset of the data
df_submission['item_cnt_month'] = best_model.predict(df_submission[features]).clip(lower=0)
save_predictions(df_submission[['shop_id', 'item_id', 'item_cnt_month']].to_pandas(), 'svm_linear_1.csv')

[I 2024-08-09 20:15:23,568] A new study created in memory with name: no-name-e9a052ec-42ff-49af-9632-52d99287fc62
[I 2024-08-09 20:16:02,751] Trial 0 finished with value: 0.9520473665751645 and parameters: {'epsilon': 0.12180038094513035, 'C': 945.1136750730599, 'penalty': 'l2', 'verbose': 0}. Best is trial 0 with value: 0.9520473665751645.
[I 2024-08-09 20:16:14,644] Trial 1 finished with value: 0.9615784884456328 and parameters: {'epsilon': 0.0789651632925978, 'C': 0.030151721755714445, 'penalty': 'l1', 'verbose': 0}. Best is trial 0 with value: 0.9520473665751645.
[I 2024-08-09 20:16:26,047] Trial 2 finished with value: 0.9661857204747637 and parameters: {'epsilon': 0.014115382079052517, 'C': 6.15767898098086, 'penalty': 'l1', 'verbose': 0}. Best is trial 0 with value: 0.9520473665751645.
[I 2024-08-09 20:16:39,074] Trial 3 finished with value: 0.9628021383203862 and parameters: {'epsilon': 0.019789763794383356, 'C': 1.5343376915338651, 'penalty': 'l1', 'verbose': 0}. Best is trial 

Best hyperparameters:  {'epsilon': 0.2187421016084914, 'C': 65.29031998207522, 'penalty': 'l1', 'verbose': 0}


ValueError: Input contains NaN.

In [28]:
from sales_project.metrics import evaluate

model = cuLinearSVR(**{'epsilon': 0.2187421016084914, 'C': 65.29031998207522, 'penalty': 'l1', 'verbose': 0})
model.fit(df_train[features], df_train[target])
df_test.loc[:, 'prediction'] = model.predict(df_test[features]).clip(lower = 0)
evaluate(df_test[[target, 'prediction']].to_pandas(), target, 'prediction')

{'MAE': 0.29447368984360023,
 'MSE': 0.6452214055867912,
 'RMSE': 0.8032567494810058,
 'R2': 0.41292622486175445,
 'MAPE': 515579225933852.7,
 'SMAPE': nan}

In [29]:
model = cuLinearSVR(**{'epsilon': 0.2187421016084914, 'C': 65.29031998207522, 'penalty': 'l1', 'verbose': 0})
model.fit(
    cudf.concat([df_train[features], df_test[features]]), 
    cudf.concat([df_train[target], df_test[target]])
)
df_submission.loc[:, 'item_cnt_month'] = model.predict(df_submission[features]).clip(lower=0)
save_predictions(df_submission[['shop_id', 'item_id', 'item_cnt_month']].to_pandas(), 'svm_linear_1.csv')

#### RandomForestRegressor -> Val: 0.83961, Test: 0.75098, Kaggle: 1.06998

In [6]:
# Setting the grid for hyperparameter optimization
hyperparameters_grid = {
    "n_estimators": {
        "method": "suggest_categorical",
        "kwargs": {"choices": [100, 200, 500]},
    },
    "max_depth": {
        "method": "suggest_int",
        "kwargs": {"low": 2, "high": 16, "step": 2},        
    },
    "n_bins": {
        "method": "suggest_int",
        "kwargs": {"low": 128, "high": 1024, "step": 128},        
    },
    "min_samples_split": {
        "method": "suggest_int",
        "kwargs": {"low": 2, "high": 100, "step": 2},        
    },
    "min_samples_leaf": {
        "method": "suggest_int",
        "kwargs": {"low": 2, "high": 100, "step": 2},        
    },
    "max_features": {
        "method": "suggest_categorical",
        "kwargs": {"choices": ['sqrt']},        
    },
    "random_state": {
        "method": "suggest_categorical",
        "kwargs": {"choices": [42]},        
    }
}

# Optimizing via Optuna
best_model, test_scores = run_optuna(
    df_train=df_train,
    df_test=df_test,
    features=features,
    target=target,
    timestamp_col='date_block_num',
    objective=time_series_objective,
    scorer=lambda y, pred: mean_squared_error(y.to_numpy(), pred.to_numpy())**(0.5),
    cv='per_timestamp',
    val_ratio=0.1,
    direction="minimize",
    n_trials=100,
    model_class = cuRF,
    hyperparameters_grid=hyperparameters_grid,
    is_gpu_accelerated=True,
    step=5,
)
print(test_scores)

# Predicting and saving predictions for the submission subset of the data
df_submission.loc[:, 'item_cnt_month'] = best_model.predict(df_submission[features]).clip(lower=0)
save_predictions(df_submission[['shop_id', 'item_id', 'item_cnt_month']].to_pandas(), 'rf_1.csv')

[I 2024-08-09 20:55:22,919] A new study created in memory with name: no-name-3152a888-fa3b-4199-b34a-398bcf302328
[I 2024-08-09 20:57:06,136] Trial 0 finished with value: 1.009993619241236 and parameters: {'n_estimators': 500, 'max_depth': 2, 'n_bins': 640, 'min_samples_split': 68, 'min_samples_leaf': 20, 'max_features': 'sqrt', 'random_state': 42}. Best is trial 0 with value: 1.009993619241236.
[I 2024-08-09 20:58:56,788] Trial 1 finished with value: 0.8775493199413708 and parameters: {'n_estimators': 200, 'max_depth': 16, 'n_bins': 256, 'min_samples_split': 58, 'min_samples_leaf': 80, 'max_features': 'sqrt', 'random_state': 42}. Best is trial 1 with value: 0.8775493199413708.
[I 2024-08-09 21:00:01,612] Trial 2 finished with value: 0.8709534847014697 and parameters: {'n_estimators': 100, 'max_depth': 16, 'n_bins': 512, 'min_samples_split': 62, 'min_samples_leaf': 72, 'max_features': 'sqrt', 'random_state': 42}. Best is trial 2 with value: 0.8709534847014697.
[I 2024-08-09 21:06:22,10

KeyboardInterrupt: 

In [7]:
from sales_project.metrics import evaluate

params = {
    "n_estimators": 200,
    "max_depth": 12,
    "n_bins": 512,
    "min_samples_split": 14,
    "min_samples_leaf": 2,
    "max_features": "sqrt",
    "random_state": 42,
}
model = cuRF(**params)
model.fit(df_train[features], df_train[target])
df_test.loc[:, "prediction"] = model.predict(df_test[features]).clip(lower=0)
evaluate(df_test[[target, "prediction"]].to_pandas(), target, "prediction")

{'MAE': 0.30265269085700586,
 'MSE': 0.5639701493125456,
 'RMSE': 0.7509794599804616,
 'R2': 0.4868550829911056,
 'MAPE': 618441953963183.4,
 'SMAPE': 184.52423317475092}

In [9]:
model = cuRF(**params)
model.fit(
    cudf.concat([df_train[features], df_test[features]]), 
    cudf.concat([df_train[target], df_test[target]])
)
df_submission.loc[:, 'item_cnt_month'] = model.predict(df_submission[features]).clip(lower=0)
save_predictions(df_submission[['shop_id', 'item_id', 'item_cnt_month']].to_pandas(), 'rf_1.csv')

#### LightGBM ->

In [4]:
import lightgbm

In [6]:
# Setting the grid for hyperparameter optimization
hyperparameters_grid = {
    "n_estimators": {
        "method": "suggest_categorical",
        "kwargs": {"choices": [100]},
    },
    "num_iterations": {
        "method": "suggest_categorical",
        "kwargs": {"choices": [100]},
    },
    "max_depth": {
        "method": "suggest_int",
        "kwargs": {"low": 1, "high": 5, "step": 1},        
    },
    "learning_rate": {
        "method": "suggest_loguniform",
        "kwargs": {"low": 1e-6, "high": 1e+0},        
    },
    # "tree_learner": {
    #     "method": "suggest_categorical",
    #     "kwargs": {"choices": ["data_parallel"]},        
    # },
    "num_threads": {
        "method": "suggest_categorical",
        "kwargs": {"choices": [10]},        
    },
    # "bagging_freq": {
    #     "method": "suggest_int",
    #     "kwargs": {"low": 5, "high": 10, "step": 1},        
    # },
    # "bagging_fraction": {
    #     "method": "suggest_discrete_uniform",
    #     "kwargs": {"low": 0.6, "high": 1., "q": 0.2},        
    # },
    "feature_fraction": {
        "method": "suggest_discrete_uniform",
        "kwargs": {"low": 0.6, "high": 1., "q": 0.2},        
    },
    "reg_alpha": {
        "method": "suggest_loguniform",
        "kwargs": {"low": 1e-6, "high": 1e+2},        
    },
    "reg_lambda": {
        "method": "suggest_loguniform",
        "kwargs": {"low": 1e-6, "high": 1e+2},           
    },
    "min_data_in_leaf": {
        "method": "suggest_int",
        "kwargs": {"low": 2, "high": 100, "step": 5},        
    },
    "random_state": {
        "method": "suggest_categorical",
        "kwargs": {"choices": [42]},        
    },
    "verbosity": {
        "method": "suggest_categorical",
        "kwargs": {"choices": [-1]},
    },
}

# Optimizing via Optuna
best_model, test_scores = run_optuna(
    df_train=df_train,
    df_test=df_test,
    features=features,
    target=target,
    objective=time_series_objective,
    scorer=lambda y, pred: mean_squared_error(y, pred)**(0.5),
    cv=5,
    val_ratio=0.1,
    direction="minimize",
    n_trials=100,
    model_class = lightgbm.LGBMRegressor,
    hyperparameters_grid=hyperparameters_grid,
    is_gpu_accelerated=False
)
print(test_scores)

# Predicting and saving predictions for the submission subset of the data
df_submission['item_cnt_month'] = best_model.predict(df_submission[features])
save_predictions(df_submission[['shop_id', 'item_id', 'item_cnt_month']], 'baseline_lightgbm_submission.csv')

[I 2024-08-04 12:11:22,132] A new study created in memory with name: no-name-0bc0cdd8-0768-4db6-b35f-ba3c10c1dd25
[I 2024-08-04 12:11:34,752] Trial 0 finished with value: 1.7958905667686431 and parameters: {'n_estimators': 100, 'num_iterations': 100, 'max_depth': 2, 'learning_rate': 7.957351180489163e-05, 'num_threads': 10, 'bagging_freq': 7, 'bagging_fraction': 0.6, 'feature_fraction': 1.0, 'reg_alpha': 1.547693867762738e-05, 'reg_lambda': 1.1849936113293921e-05, 'min_data_in_leaf': 47, 'random_state': 42, 'verbosity': -1}. Best is trial 0 with value: 1.7958905667686431.
[I 2024-08-04 12:11:47,859] Trial 1 finished with value: 1.7564493688842753 and parameters: {'n_estimators': 100, 'num_iterations': 100, 'max_depth': 2, 'learning_rate': 0.0012049686168336979, 'num_threads': 10, 'bagging_freq': 7, 'bagging_fraction': 0.8, 'feature_fraction': 1.0, 'reg_alpha': 0.013589623579568564, 'reg_lambda': 0.0009211838162923209, 'min_data_in_leaf': 82, 'random_state': 42, 'verbosity': -1}. Best i

Best hyperparameters:  {'n_estimators': 100, 'num_iterations': 100, 'max_depth': 5, 'learning_rate': 0.14532674639727586, 'num_threads': 10, 'bagging_freq': 5, 'bagging_fraction': 1.0, 'feature_fraction': 0.6, 'reg_alpha': 2.027926520134932, 'reg_lambda': 0.0004014233367768531, 'min_data_in_leaf': 27, 'random_state': 42, 'verbosity': -1}
{'MAE': 0.38823887315145494, 'MSE': 6.818552084556274, 'RMSE': 2.611235738985715, 'R2': 0.2069697177742893, 'MAPE': 837576660394750.1, 'SMAPE': 185.92091845468065}


AttributeError: 'DataFrame' object has no attribute 'to_pandas'

In [5]:
# Setting the grid for hyperparameter optimization
hyperparameters_grid = {
    "n_estimators": {
        "method": "suggest_categorical",
        "kwargs": {"choices": [100]},
    },
    "num_iterations": {
        "method": "suggest_categorical",
        "kwargs": {"choices": [100]},
    },
    "max_depth": {
        "method": "suggest_int",
        "kwargs": {"low": 1, "high": 10, "step": 1},        
    },
    "learning_rate": {
        "method": "suggest_loguniform",
        "kwargs": {"low": 1e-6, "high": 1e+0},        
    },
    # "tree_learner": {
    #     "method": "suggest_categorical",
    #     "kwargs": {"choices": ["data_parallel"]},        
    # },
    "num_threads": {
        "method": "suggest_categorical",
        "kwargs": {"choices": [10]},        
    },
    # "bagging_freq": {
    #     "method": "suggest_int",
    #     "kwargs": {"low": 5, "high": 10, "step": 1},        
    # },
    # "bagging_fraction": {
    #     "method": "suggest_discrete_uniform",
    #     "kwargs": {"low": 0.6, "high": 1., "q": 0.2},        
    # },
    "feature_fraction": {
        "method": "suggest_discrete_uniform",
        "kwargs": {"low": 0.6, "high": 1., "q": 0.2},        
    },
    "reg_alpha": {
        "method": "suggest_loguniform",
        "kwargs": {"low": 1e-6, "high": 1e+2},        
    },
    "reg_lambda": {
        "method": "suggest_loguniform",
        "kwargs": {"low": 1e-6, "high": 1e+2},           
    },
    "min_data_in_leaf": {
        "method": "suggest_int",
        "kwargs": {"low": 2, "high": 100, "step": 5},        
    },
    "random_state": {
        "method": "suggest_categorical",
        "kwargs": {"choices": [42]},        
    },
    "verbosity": {
        "method": "suggest_categorical",
        "kwargs": {"choices": [-1]},
    },
}

# Optimizing via Optuna
best_model, test_scores = run_optuna(
    df_train=df_train,
    df_test=df_test,
    features=features,
    target=target,
    objective=time_series_objective,
    scorer=lambda y, pred: mean_squared_error(y, pred)**(0.5),
    cv=5,
    val_ratio=0.1,
    direction="minimize",
    n_trials=100,
    model_class = lightgbm.LGBMRegressor,
    hyperparameters_grid=hyperparameters_grid,
    is_gpu_accelerated=False
)
print(test_scores)

# Predicting and saving predictions for the submission subset of the data
df_submission['item_cnt_month'] = best_model.predict(df_submission[features])
save_predictions(df_submission[['shop_id', 'item_id', 'item_cnt_month']], 'baseline_lightgbm_submission2.csv')

[I 2024-08-04 12:41:26,019] A new study created in memory with name: no-name-2022c9a2-e263-47c1-b1e1-6e304e65b8eb
[I 2024-08-04 12:41:43,848] Trial 0 finished with value: 1.4822324821626158 and parameters: {'n_estimators': 100, 'num_iterations': 100, 'max_depth': 8, 'learning_rate': 0.014518871232101084, 'num_threads': 10, 'feature_fraction': 0.6, 'reg_alpha': 0.3115135343837188, 'reg_lambda': 0.0009309693480445781, 'min_data_in_leaf': 27, 'random_state': 42, 'verbosity': -1}. Best is trial 0 with value: 1.4822324821626158.
[I 2024-08-04 12:42:00,474] Trial 1 finished with value: 1.785002731788285 and parameters: {'n_estimators': 100, 'num_iterations': 100, 'max_depth': 10, 'learning_rate': 0.00029022556084904474, 'num_threads': 10, 'feature_fraction': 0.8, 'reg_alpha': 0.010739501583809796, 'reg_lambda': 0.00015084324151151037, 'min_data_in_leaf': 92, 'random_state': 42, 'verbosity': -1}. Best is trial 0 with value: 1.4822324821626158.
[I 2024-08-04 12:42:17,153] Trial 2 finished with

Best hyperparameters:  {'n_estimators': 100, 'num_iterations': 100, 'max_depth': 10, 'learning_rate': 0.037369863707880736, 'num_threads': 10, 'feature_fraction': 1.0, 'reg_alpha': 0.0023523282514064567, 'reg_lambda': 0.06189621210183282, 'min_data_in_leaf': 12, 'random_state': 42, 'verbosity': -1}
{'MAE': 0.3951232671911021, 'MSE': 7.1316534305533175, 'RMSE': 2.6705155739207584, 'R2': 0.17055453083985617, 'MAPE': 853048010748982.5, 'SMAPE': 186.24473011418849}
